In [ ]:
# A library to avoid Python warnings
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
# # Install a library to aviod Tensorflow warnings
# ! pip install -q silence_tensorflow  

# from silence_tensorflow import silence_tensorflow
# silence_tensorflow()

# print()

In [ ]:
import os
import random as rn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [ ]:
if len(tf.config.list_physical_devices('GPU')) == 0 :
    print(f"CPU is in used")
else :
    print(f"GPU is in used")

In [ ]:
print(f'Tensorflow Version : {tf.__version__}')

In [ ]:
root_dir = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset' # root directory
f_names = [f_name for f_name in os.listdir(root_dir)] # names of folders (or files)
print(f"Folder (or file) names:\n{f_names}")

In [ ]:
f_names.remove('Rice_Citation_Request.txt')

In [ ]:
rice_classes = f_names.copy()
print(f"Rice classes:\n{', '.join(rice_classes)}")

In [ ]:
from PIL import Image

def samples_rice_images(rice_class):
    rice_dir = os.listdir(os.path.join(root_dir, rice_class))
    random_selection = rn.choices(rice_dir, k=5)

    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    for i, img_name in enumerate(random_selection):
        img_path = os.path.join(root_dir, rice_class, img_name)
        img = Image.open(img_path)  
        axes[i].imshow(img)
        axes[i].axis('off')

    plt.show()

<div style="text-align: center; background-color: #314644; padding: 10px; border-radius: 1px; box-shadow: 0 0 0px rgba(0, 0, 0, 0.5);border-radius:60px;">
    <span style="font-family: 'Roboto Condensed', sans-serif; font-size: 20px;color:#FFFF00;">Karacadag</span>
</div>


In [ ]:
samples_rice_images("Karacadag")

<div style="text-align: center; background-color: #314644; padding: 10px; border-radius: 1px; box-shadow: 0 0 0px rgba(0, 0, 0, 0.5);border-radius:60px;">
    <span style="font-family: 'Roboto Condensed', sans-serif; font-size: 20px;color:#FFFF00;">Basmati</span>
</div>


In [ ]:
samples_rice_images("Basmati")

<div style="text-align: center; background-color: #314644; padding: 10px; border-radius: 1px; box-shadow: 0 0 0px rgba(0, 0, 0, 0.5);border-radius:60px;">
    <span style="font-family: 'Roboto Condensed', sans-serif; font-size: 20px;color:#FFFF00;">Jasmine</span>
</div>


In [ ]:
samples_rice_images("Jasmine")

<div style="text-align: center; background-color: #314644; padding: 10px; border-radius: 1px; box-shadow: 0 0 0px rgba(0, 0, 0, 0.5);border-radius:60px;">
    <span style="font-family: 'Roboto Condensed', sans-serif; font-size: 20px;color:#FFFF00;">Arborio</span>
</div>


In [ ]:
samples_rice_images("Arborio")

<div style="text-align: center; background-color: #314644; padding: 10px; border-radius: 1px; box-shadow: 0 0 0px rgba(0, 0, 0, 0.5);border-radius:60px;">
    <span style="font-family: 'Roboto Condensed', sans-serif; font-size: 20px;color:#FFFF00;">Ipsala</span>
</div>


In [ ]:
samples_rice_images("Ipsala")

In [ ]:
# Create a dictionary that the key is the rice class and the value is the number of each classes in it
classes_counts = {rice_class: len(os.listdir(os.path.join(root_dir, rice_class))) for rice_class in rice_classes}

In [ ]:
sizes = list(classes_counts.values())
labels = list(classes_counts.keys())

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=sizes,
                             hole=0.3,
                             marker_colors=px.colors.sequential.Greens,
                             textinfo='label+percent',
                             insidetextorientation='radial')])

fig.update_layout(title='Rice Classes Distribution')
fig.show()

In [ ]:
rice_list = []

for rice_class in rice_classes:
    class_dir = os.path.join(root_dir, rice_class)
    for img_path in os.listdir(class_dir):
        full_path = os.path.join(class_dir, img_path)
        rice_list.append((full_path, rice_class))

In [ ]:
# Dataframe Before Shuffling
df = pd.DataFrame(rice_list, columns=["File_Path", "Label"])

In [ ]:
df.head(10)

In [ ]:
# Dataframe After Shuffling
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head(10)

In [ ]:
total_samples = df.shape[0]
print(f"Total number of samples is {total_samples}")

In [ ]:
0.15 * total_samples, 0.7 * total_samples

In [ ]:
train_set = df[:52500].copy() # 70% of data
validate_set = df[52500:63750].copy()  # 15% of data
test_set = df[63750:].copy() # 15% of data

In [ ]:
def train_calidate_test_plot():
    colors = ['#4CAF50', '#7CB342', '#CDDC39', '#FFEB3B', '#FFC107', '#FF9800', '#FF5722', '#F44336', '#E91E63', '#9C27B0']

    category_counts = df['Label'].value_counts()
    categories = category_counts.index.tolist()
    counts = category_counts.values.tolist()

    fig, axs = plt.subplots(1, 3, figsize=(15, 5))

    for i, (subset, subset_name) in enumerate(zip([train_set, validate_set, test_set], ['Training Set', 'Validation Set', 'Test Set'])):
        subset_category_counts = subset['Label'].value_counts()
        subset_counts = subset_category_counts.values.tolist()

        total_count = len(subset)
        percentages = [count / total_count * 100 for count in subset_counts]

        wedges, _, autotexts = axs[i].pie(subset_counts, labels=subset_category_counts.index, autopct=lambda p: f'{p:.1f}%\n({total_count * p / 100:.0f})', colors=colors, textprops={'color': 'black', 'fontsize': 10})
        axs[i].set_title(f'{subset_name} ({total_count})')

        for autotext in autotexts:
            autotext.set_horizontalalignment('center')
            autotext.set_verticalalignment('center')

    plt.tight_layout()
    plt.show()

In [ ]:
train_calidate_test_plot()

In [ ]:
image_path = df["File_Path"][0]
image = Image.open(image_path)

width, height = image.size

print("Check the size of a sample image")
print(f"{df['Label'][0]} -> Width: {width}, Height: {height}")

In [ ]:
epochs = 5
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_dataframe(
  dataframe=train_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

validation_generator = data_generator.flow_from_dataframe(
  dataframe=validate_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = data_generator.flow_from_dataframe(
  dataframe=test_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

In [ ]:
for image_batch, labels_batch in train_generator:
    print(f"Train Shape: {image_batch.shape} (Batches = {len(train_generator)})")
    print(f"Train label: {labels_batch.shape}\n")
    break
    
for image_batch, labels_batch in validation_generator:
    print(f"Validation Shape: {image_batch.shape} (Batches = {len(validation_generator)})")
    print(f"Validation label: {labels_batch.shape}\n")
    break
    
for image_batch, labels_batch in test_generator:
    print(f"Test Shape: {image_batch.shape} (Batches = {len(test_generator)})")
    print(f"Test label: {labels_batch.shape}\n")
    break

In [ ]:
CNN_model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')  
])


In [ ]:
plot_model(CNN_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, dpi=100, expand_nested=True)

In [ ]:
CNN_model.summary()

In [ ]:
CNN_model.compile(loss=BinaryCrossentropy(),
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

In [ ]:
CNN = CNN_model.fit(train_generator,
                    epochs=epochs,
                    validation_data=validation_generator)

In [ ]:
CNN_model.save('CNN_model.h5')
history_CNN = CNN.history

In [ ]:
AlexNet_model = models.Sequential([
    layers.Conv2D(96, 11, strides=4, activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(256, 5, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(384, 3, activation='relu'),
    layers.Conv2D(384, 3, activation='relu'),
    layers.Conv2D(256, 3, activation='relu'),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

In [ ]:
plot_model(AlexNet_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, dpi=100, expand_nested=True)

In [ ]:
AlexNet_model.summary()

In [ ]:
AlexNet_model.compile(loss=BinaryCrossentropy(),
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

In [ ]:
AlexNet = AlexNet_model.fit(train_generator,
                    epochs=epochs,
                    validation_data=validation_generator)

In [ ]:
AlexNet_model.save('AlexNet_model.h5')
history_alex = AlexNet.history

In [ ]:
def eval_result(Model1, history1, Model2, history2):
    fig, axs = plt.subplots(2, 2, figsize=(20, 10))

    axs[0, 0].plot(history1['accuracy'], color='green', label=f'{Model1} Train Accuracy')
    axs[0, 0].plot(history2['accuracy'], '--', color='blue', label=f'{Model2} Train Accuracy')
    axs[0, 0].set_title('Train Accuracy Comparison')
    axs[0, 0].set_xlabel('Epoch')
    axs[0, 0].set_ylabel('Accuracy')
    axs[0, 0].legend(loc='best')
    axs[0, 0].tick_params(axis='both', which='both', direction='in', labelsize=12)

    axs[0, 1].plot(history1['val_accuracy'], color='green', label=f'{Model1} Validation Accuracy')
    axs[0, 1].plot(history2['val_accuracy'], '--', color='blue', label=f'{Model2} Validation Accuracy')
    axs[0, 1].set_title('Validation Accuracy Comparison')
    axs[0, 1].set_xlabel('Epoch')
    axs[0, 1].set_ylabel('Accuracy')
    axs[0, 1].legend(loc='best')
    axs[0, 1].tick_params(axis='both', which='both', direction='in', labelsize=12)

    axs[1, 0].plot(history1['loss'], color='green', label=f'{Model1} Train Loss')
    axs[1, 0].plot(history2['loss'], '--', color='blue', label=f'{Model2} Train Loss')
    axs[1, 0].set_title('Train Loss Comparison')
    axs[1, 0].set_xlabel('Epoch')
    axs[1, 0].set_ylabel('Loss')
    axs[1, 0].legend(loc='best')
    axs[1, 0].tick_params(axis='both', which='both', direction='in', labelsize=12)

    axs[1, 1].plot(history1['val_loss'], color='green', label=f'{Model1} Validation Loss')
    axs[1, 1].plot(history2['val_loss'], '--', color='blue', label=f'{Model2} Validation Loss')
    axs[1, 1].set_title('Validation Loss Comparison')
    axs[1, 1].set_xlabel('Epoch')
    axs[1, 1].set_ylabel('Loss')
    axs[1, 1].legend(loc='best')
    axs[1, 1].tick_params(axis='both', which='both', direction='in', labelsize=12)

    plt.tight_layout()
    plt.show()

In [ ]:
eval_result("CNN", history_CNN, "AlexNet", history_alex)

In [ ]:
load_CNN = tf.keras.models.load_model('CNN_model.h5')
load_Alex = tf.keras.models.load_model('AlexNet_model.h5')

In [ ]:
def plot_confusion_matrix(conf_matrix, class_labels, model_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix of {model_name}')
    plt.show()

def print_classification_report(true_classes, predicted_classes, class_labels, model_name):
    print(f"Classification Report for {model_name} Model:\n")
    print(classification_report(true_classes, predicted_classes, target_names=class_labels))

def evaluate_model(load_model, test_generator, class_labels, model_name):
    predictions = load_model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    
    return conf_matrix, true_classes, predicted_classes

conf_matrix_CNN, true_classes_CNN, predicted_classes_CNN = evaluate_model(load_CNN, test_generator, rice_classes, "CNN")
conf_matrix_AlexNet, true_classes_AlexNet, predicted_classes_AlexNet = evaluate_model(load_Alex, test_generator, rice_classes, "AlexNet")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8))

sns.heatmap(conf_matrix_CNN, annot=True, fmt='d', cmap='Greens', xticklabels=rice_classes, yticklabels=rice_classes, ax=axs[0])
axs[0].set_xlabel('Predicted Labels')
axs[0].set_ylabel('True Labels')
axs[0].set_title('Confusion Matrix of CNN Model')

sns.heatmap(conf_matrix_AlexNet, annot=True, fmt='d', cmap='Greens', xticklabels=rice_classes, yticklabels=rice_classes, ax=axs[1])
axs[1].set_xlabel('Predicted Labels')
axs[1].set_ylabel('True Labels')
axs[1].set_title('Confusion Matrix of AlexNet Model')

plt.tight_layout()
plt.show()

In [ ]:
print_classification_report(true_classes_CNN, predicted_classes_CNN, rice_classes, "CNN")
print(53*"*")
print_classification_report(true_classes_AlexNet, predicted_classes_AlexNet, rice_classes, "AlexNet")

<div style="background-color:#314644; padding: 20px; border-radius: 10px; box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1);">
    <div style="background-color:#6A8D84; padding: 20px; border-radius: 10px; box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1);">
        <table style="font-size:20px; font-family:calibri; line-height: 1.5em; margin: 0 auto; width: 100%;">
            <tr>
                <td rowspan="2" style="font-weight:bold; text-align:center;">Class</td>
                <td colspan="4" style="font-weight:bold; text-align:center;">CNN</td>
                <td colspan="4" style="font-weight:bold; text-align:center;">AlexNet</td>
            </tr>
            <tr>
                <td style="font-weight:bold; text-align:center;">Precision</td>
                <td style="font-weight:bold; text-align:center;">Recall</td>
                <td style="font-weight:bold; text-align:center;">F1-score</td>
                <td style="font-weight:bold; text-align:center;">Support</td>
                <td style="font-weight:bold; text-align:center;">Precision</td>
                <td style="font-weight:bold; text-align:center;">Recall</td>
                <td style="font-weight:bold; text-align:center;">F1-score</td>
                <td style="font-weight:bold; text-align:center;">Support</td>
            </tr>
            <tr>
                <td style="text-align:left;">Karacadag</td>
                <td style="text-align:center;">0.98</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">2268</td>
                <td style="text-align:center;">0.87</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">0.92</td>
                <td style="text-align:center;">2268</td>
            </tr>
            <tr>
                <td style="text-align:left;">Basmati</td>
                <td style="text-align:center;">1.00</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">2265</td>
                <td style="text-align:center;">0.98</td>
                <td style="text-align:center;">0.97</td>
                <td style="text-align:center;">0.97</td>
                <td style="text-align:center;">2265</td>
            </tr>
            <tr>
                <td style="text-align:left;">Jasmine</td>
                <td style="text-align:center;">0.97</td>
                <td style="text-align:center;">1.00</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">2204</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">1.00</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">2204</td>
            </tr>
            <tr>
                <td style="text-align:left;">Arborio</td>
                <td style="text-align:center;">0.98</td>
                <td style="text-align:center;">0.97</td>
                <td style="text-align:center;">0.98</td>
                <td style="text-align:center;">2274</td>
                <td style="text-align:center;">0.97</td>
                <td style="text-align:center;">0.96</td>
                <td style="text-align:center;">0.96</td>
                <td style="text-align:center;">2274</td>
            </tr>
            <tr>
                <td style="text-align:left;">Ipsala</td>
                <td style="text-align:center;">1.00</td>
                <td style="text-align:center;">0.98</td>
                <td style="text-align:center;">0.99</td>
                <td style="text-align:center;">2239</td>
                <td style="text-align:center;">1.00</td>
                <td style="text-align:center;">0.85</td>
                <td style="text-align:center;">0.92</td>
                <td style="text-align:center;">2239</td>
            </tr>
            <tr>
                <td style="font-weight:bold; text-align:center;">Accuracy</td>
                <td colspan="4" style="text-align:center;">0.99</td>
                <td colspan="4" style="text-align:center;">0.95</td>
            </tr>
        </table>
    </div>
</div>


In [ ]:
test_generator.reset()

images, labels = next(test_generator)

predictions = CNN_model.predict(images)
predicted_classes = np.argmax(predictions, axis=1)

plt.figure(figsize=[14, 14])
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(images[i])
    plt.axis('off')
    plt.title("Label: {}\nPrediction: {} ({:.1f}%)".format(
        rice_classes[np.argmax(labels[i])], 
        rice_classes[predicted_classes[i]], 
        100 * np.max(predictions[i])
    ))
    
plt.show()    

In [ ]:
import os
import shutil

source_dir = "/kaggle/input/rice-image-dataset/Rice_Image_Dataset"               # original dataset
target_dir = "/kaggle/working/"             # output directory
samples_per_class = 10                   # number of images to save from each class

os.makedirs(target_dir, exist_ok=True)

for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        output_class_path = os.path.join(target_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)
        
        images = os.listdir(class_path)[:samples_per_class]
        for img in images:
            src = os.path.join(class_path, img)
            dst = os.path.join(output_class_path, img)
            shutil.copyfile(src, dst)

print("Saved sample images per label in:", target_dir)


In [ ]:
import shutil

# Zip the model file
shutil.make_archive('/kaggle/working/AlexNet_model', 'zip', '/kaggle/working', 'AlexNet_model.h5')
